"""
# Proyecto MLOps: Modelo predictivo de ventas - Evaluación de para tienda Minorista

Este proyecto forma parte de una pipeline de desarrollo y despliegue de modelos de machine learning en entornos productivos. El objetivo es construir un modelo robusto capaz de predecir el total de ventas mensuales de una tienda minorista, a partir de datos históricos diarios.

El sistema se integrará en una arquitectura MLOps donde los artefactos del modelo, la trazabilidad de experimentos, y las métricas estarán disponibles para auditoría y versionado.

-------------------------------------------------------------------------------
🔧 FASES DEL DESARROLLO DEL MODELO

## 1. Ingesta y Preparación de Datos
- Fuente: CSV / base de datos.
- Procesos:
    - Validación de esquema y tipos de datos.
    - Imputación de valores faltantes.
    - Codificación de variables categóricas si aplica.
    - División en conjuntos de entrenamiento, validación y test.

## 2. Análisis Exploratorio de Datos (EDA)
- Análisis visual y estadístico de distribución de ventas.
- Evaluación de correlaciones entre variables predictoras y objetivo.
- Agrupamientos temporales: comparativas entre días hábiles, festivos y fines de semana.
- Identificación de outliers o patrones estacionales.

## 3. Entrenamiento de Modelos Base
- Modelos evaluados:
    - `LinearRegression()`
    - `DecisionTreeRegressor()`
    - `RandomForestRegressor()`
    - *(opcional: agregar Lasso, XGBoost, etc.)*
- Métricas:
    - Coeficiente de determinación: R²
    - Error cuadrático medio (RMSE)
    - Error absoluto medio (MAE)
- Técnicas:
    - Búsqueda de hiperparámetros (`GridSearchCV` / `RandomizedSearchCV`)
    - Validación cruzada (`cross_val_score`)

## 4. Evaluación y Visualización
- Generación de reportes con métricas para cada modelo.
- Gráficos de comparación: ventas reales vs predichas.
- Análisis de errores por categoría y por horizonte temporal.

## 5. Empaquetado del Modelo
- Serialización del mejor modelo (`joblib` o `pickle`).
- Exportación de métricas y visualizaciones.
- Generación de archivo `requirements.txt` y documentación del entorno virtual.

## 6. Preparación para Producción (MLOps)
- Scripts de inferencia reutilizables (`predict.py`)
- Interfaz REST o CLI para consultas.
- Documentación técnica y funcional en README.md
- (Opcional) Integración con `MLflow` para seguimiento de experimentos.
- (Opcional) Despliegue en contenedor Docker / pipeline CI/CD.

-------------------------------------------------------------------------------
🎯 OBJETIVO DEL MODELO
Predecir el total de ventas para el mes siguiente en función de variables históricas,
lo que permitirá a la tienda:
- Optimizar su inventario.
- Definir estrategias promocionales.
- Asignar personal de forma eficiente.

-------------------------------------------------------------------------------
🧾 NOTAS ADICIONALES
- Enfocado en reproducibilidad, modularidad y trazabilidad.
- Compatible con flujos de trabajo de ciencia de datos y arquitectura MLOps.
- Código limpio, comentado y preparado para pruebas automatizadas.

"""


## 1. Preparación de datos

In [1]:
# Importar librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [2]:
# Datos de la tienda minorista
ruta = "./Datos/Ventas.csv"

# Cargar el dataset
df = pd.read_csv(ruta)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './Datos/Ventas.csv'

In [ ]:
# Comprobar si hay valores faltantes
df.info()

In [ ]:
# Convertir la columna 'Fecha' a tipo datetime
df['Fecha'] = pd.to_datetime(df['Fecha'])
df.head()

In [ ]:
escala = MinMaxScaler(feature_range=(0, 1))

# Eliminamos las columnas no deseadas y guardamos los nombres de las columnas que vamos a escalar
columnas_para_escalar = df.drop(["Ventas", "Fecha"], axis=1).columns

# Ajustamos el MinMaxScaler a las columnas restantes y transformamos los datos
normado = escala.fit_transform(df[columnas_para_escalar])

# Creamos un nuevo DataFrame con los datos normalizados y las columnas correctas
df_normado = pd.DataFrame(data=normado, columns=columnas_para_escalar)

# Si necesitas, puedes agregar las columnas no escaladas que has quitado previamente
df_normado["Ventas"] = df["Ventas"]
df_normado["Fecha"] = df["Fecha"]

# Ahora df_normado tiene los datos normalizados y las columnas no normalizadas originales
df_normado.head()

In [ ]:
# Comprobación del tipo de dato de Fecha
df.info()

In [ ]:
# Identificar las variables dependientes e independientes
X = df_normado.drop(['Ventas', 'Fecha'], axis=1)
y = df['Ventas']

In [ ]:
# Dividir en conjuntos de entrenamiento y prueba
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(X, y, train_size=0.8, random_state=42)

## 2. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Estadísticas descriptivas
df.describe()

In [ ]:
# Visualización de la distribución de ventas
plt.figure(figsize=(12, 6))
sns.histplot(df['Ventas'], bins=30, kde=True)
plt.title('Distribución de Ventas')
plt.xlabel('Ventas')
plt.ylabel('Frecuencia');

In [ ]:
# Relación entre ventas y día de la semana con promedio de ventas
plt.figure(figsize=(12, 6))
sns.barplot(data=df, x='DíaDeLaSemana', y='Ventas', hue='DíaDeLaSemana', palette='viridis')
plt.title('Ventas promedio por Día de la Semana')
plt.xlabel('Día de la Semana')
plt.ylabel('Ventas Promedio');

In [ ]:
# boxplot para comparar las ventas con y sin promociones.
plt.figure(figsize=(12, 6))
sns.boxplot(x='Promociones', y='Ventas', data=df, hue='Promociones')
plt.title('Efecto de las Promociones en las Ventas')

# boxplot para comparar las ventas para los días normales y festivos.
plt.figure(figsize=(12, 6))
sns.boxplot(x='Festivo', y='Ventas', data=df,  hue='Festivo')
plt.title('Efecto de los Días Festivos en las Ventas')

# boxplot para ver la interacción entre promociones y días festivos en las ventas.
plt.figure(figsize=(12, 6))
sns.boxplot(x='Promociones', y='Ventas',  hue='Promociones', data=df)
plt.title('Interacción entre Promociones y Días Festivos en las Ventas');

## 3. Selección de modelo

In [ ]:
# almacenar modelos
modelos = [
    ("modelo lineal", LinearRegression()),
    ("modelo arbol", DecisionTreeRegressor(random_state=42)),
    ("modelo bosque", RandomForestRegressor(random_state=42))
]

In [ ]:
for nombre, modelo in modelos:
    modelo.fit(X_entrena, y_entrena)
    pred = modelo.predict(X_prueba)
    r2 = r2_score(y_prueba, pred)
    rmse = np.sqrt(mean_squared_error(y_prueba, pred))  # corregido
    print(f'{nombre}: R² = {r2:.3f}, RMSE = {rmse:.2f}')


## 4. Entrenamiento y evaluación del modelo

In [ ]:
# Alojamos el modelo de Regresión Lineal en una variable
modelo_lineal = LinearRegression()

# Entrenamos el modelo con los datos de entrenamiento
modelo_lineal.fit(X_entrena, y_entrena)

# Realizamos predicciones usando el conjunto de prueba
predicciones_lineal = modelo_lineal.predict(X_prueba)

In [ ]:
# Alojamos el modelo de Regresión Lineal en una variable
modelo_lineal = LinearRegression()
modelo_lineal.fit(X_entrena, y_entrena)
predicciones_lineal = modelo_lineal.predict(X_prueba)

# Cálculo de métricas
r2 = r2_score(y_prueba, predicciones_lineal)
rmse = np.sqrt(mean_squared_error(y_prueba, predicciones_lineal))

# Gráfico de dispersión real vs predicho
plt.figure(figsize=(10, 6))
plt.scatter(y_prueba, predicciones_lineal, alpha=0.5)
plt.plot([y_prueba.min(), y_prueba.max()], [y_prueba.min(), y_prueba.max()], linestyle='--')
plt.xlabel('Ventas Reales')
plt.ylabel('Ventas Predichas')
plt.title(f'Modelo Lineal\n$R^2$ = {r2:.3f}, RMSE = {rmse:.2f}')
plt.grid(True)
plt.show()



In [ ]:
df_test = pd.DataFrame({'Real': y_prueba, 'Predicho': predicciones_lineal})
df_test = df_test.sort_index()

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(df_test['Real'], label='Ventas Reales', alpha=0.7)
plt.plot(df_test['Predicho'], label='Ventas Predichas', alpha=0.7)
plt.legend()
plt.title('Comparación de Ventas Reales y Ventas Predichas a lo largo del tiempo');

## 5. Conclusión

1. El primer gráfico, que muestra un diagrama de dispersión de las Ventas Reales vs Ventas Predichas, sugiere que el modelo de regresión lineal está haciendo un buen trabajo al predecir las ventas. La línea de tendencia indica una fuerte relación positiva entre los valores reales y predichos, lo que es un signo prometedor de que el modelo puede capturar la tendencia de las ventas con eficacia.
2. El segundo gráfico compara las Ventas Reales y las Ventas Predichas a lo largo del tiempo y también parece seguir un patrón similar, aunque hay algunos puntos en los que las predicciones y los valores reales difieren significativamente. Esto puede deberse a eventos no capturados por las variables en tu modelo o a variaciones naturales en las ventas que no son predecibles.

Aquí hay algunas recomendaciones para la tienda minorista:
* Optimización de Inventario: Utiliza las predicciones para gestionar mejor el inventario. Las fechas festivas pueden requerir un stock adicional para evitar la falta de productos.
* Planificación de Personal: Ajusta los horarios del personal según días festivos, y no necesariamente según dias de promociones. 
* Marketing Dirigido: Si identificas patrones de cuándo las ventas son más fuertes, puedes dirigir las campañas de marketing para esos periodos y potencialmente aumentar aún más las ventas.
* Análisis de Anomalías: Investiga aquellos puntos donde hay grandes desviaciones entre las ventas reales y las predichas para entender mejor los factores no capturados por el modelo.
* Mejoras en el Modelo: Considera incluir más variables en el modelo que puedan afectar las ventas, como datos económicos generales, eventos locales, competencia, o incluso el clima.
